In [ ]:
# default_exp models.cox

# Cox Proportional Hazard
> SA with features apart from time

$$
\begin{aligned}
\lambda(t,x) = \lambda(t)h(x)
\end{aligned}
$$

In [ ]:
# export
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchlife.losses import hazard_loss
from torchlife.models.ph import PieceWiseHazard

from sklearn.preprocessing import MaxAbsScaler, StandardScaler

# torch.Tensor.ndim = property(lambda x: x.dim())

In [ ]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

$$
\begin{aligned}
\int_0^T \lambda(t,x) dt &= \int_0^T \lambda(t)h(x) dt\\
&= h(x)\int_0^T \lambda(t) dt
\end{aligned}
$$

In [ ]:
# export
class ProportionalHazard(nn.Module):
    """
    Hazard proportional to time and feature component as shown above.
    parameters:
    - breakpoints: time points where hazard would change
    - max_t: maximum point of time to plot to.
    - dim: number of input dimensions of x
    - h: (optional) number of hidden units (for x only).
    """
    def __init__(self, breakpoints:np.array, t_scaler:MaxAbsScaler, x_scaler:StandardScaler, 
                 dim:int, h:tuple=(), **kwargs):
        super().__init__()
        self.baseλ = PieceWiseHazard(breakpoints, t_scaler)
        self.x_scaler = x_scaler
        nodes = (dim,) + h + (1,)
        self.layers = nn.ModuleList([nn.Linear(a,b, bias=False) 
                                   for a,b in zip(nodes[:-1], nodes[1:])])
        
    def forward(self, t, t_section, x):
        logλ, Λ = self.baseλ(t, t_section)
        
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        log_hx = self.layers[-1](x)
        
        logλ += log_hx
        Λ = torch.exp(log_hx + torch.log(Λ))
        return logλ, Λ
    
    def survival_function(self, t, x):
        if len(t.shape) == 1:
            t = t[:,None]
        t = self.baseλ.t_scaler.transform(t)
        if len(x.shape) == 1:
            x = x[:,None]
        x = self.x_scaler.transform(x)
        
        with torch.no_grad():
            x = torch.Tensor(x)
            # get the times and time sections for survival function
            breakpoints = self.baseλ.breakpoints[1:].cpu().numpy()
            t_sec_query = np.searchsorted(breakpoints.squeeze(), t.squeeze())
            # convert to pytorch tensors
            t_query = torch.Tensor(t)[:,None]
            t_sec_query = torch.LongTensor(t_sec_query)

            # calculate cumulative hazard according to above
            _, Λ = self.forward(t_query, t_sec_query, x)
            return torch.exp(-Λ)
            
    
    def plot_survival_function(self, t, x):
        s = self.survival_function(t, x)
        
        # plot
        plt.figure(figsize=(12,5))
        plt.plot(t, s)
        plt.xlabel('Time')
        plt.ylabel('Survival Probability')
        plt.show()

## Fitting Cox Proportional Hazard Model

In [ ]:
# hide
from torchlife.data import create_db, get_breakpoints
import pandas as pd

In [ ]:
# hide
url = "https://raw.githubusercontent.com/CamDavidsonPilon/lifelines/master/lifelines/datasets/rossi.csv"
df = pd.read_csv(url)
df.head()

,week,arrest,fin,age,race,wexp,mar,paro,prio
0,20,1,0,27,1,0,0,1,3
1,17,1,0,18,1,0,0,1,8
2,25,1,0,19,0,1,0,1,13
3,52,0,1,23,1,1,1,1,1
4,52,0,0,19,0,1,0,1,3


In [ ]:
# hide
df.rename(columns={'week':'t', 'arrest':'e'}, inplace=True)
breakpoints, widths = get_breakpoints(df)
db = create_db(df, breakpoints)

ValueError: too many values to unpack (expected 2)

In [ ]:
# hide
from fastai.basics import Learner

x_dim = df.shape[1] - 2
model = ProportionalHazard(breakpoints, widths, x_dim, h=(3,3))
learner = Learner(db, model, loss_func=hazard_loss)
# wd = 1e-4
learner.lr_find()
learner.recorder.plot()

In [ ]:
# hide
epochs = 10
learner.fit(epochs, lr=1)

## Plotting hazard functions

In [ ]:
model.baseλ.plot_hazard()
x = df.drop(['t', 'e'], axis=1).iloc[4]
t = np.arange(df['t'].max())
model.plot_survival_function(t, x)

In [ ]:
# hide
from nbdev.export import *
notebook2script()